# Лабораторная работа 1

Выполнили студенты 25ИАД:
1) Данил Ерёменко
2) Кудасов Максим

## Подготовка данных

In [197]:
from sklearn.decomposition import TruncatedSVD
import numpy as np
import pandas as pd
import copy

In [109]:
# Считываем датасет
path_to_file = "Materials/Bank_Example.xlsx"
X_data = pd.read_excel(path_to_file, header=None).to_numpy()
X_data

array([[9, 6, 9, 2, 2],
       [4, 6, 2, 6, 7],
       [0, 0, 5, 0, 0],
       [2, 2, 0, 9, 9],
       [6, 9, 8, 3, 3],
       [3, 8, 5, 4, 7],
       [4, 5, 6, 3, 6],
       [8, 6, 8, 2, 2],
       [4, 4, 0, 8, 8],
       [2, 8, 4, 5, 7],
       [1, 2, 6, 0, 0],
       [6, 9, 7, 3, 5],
       [6, 7, 1, 7, 8],
       [2, 1, 7, 1, 1],
       [9, 7, 9, 2, 1]])

In [110]:
# Центрируем и нормализуем (через L2-норму)
X_centered = X_data - X_data.mean(axis=0)
X_cleaned = X_centered / np.linalg.norm(X_centered, axis=0)
X_cleaned

array([[ 0.43158731,  0.06102782,  0.33188947, -0.15939418, -0.19889806],
       [-0.03752933,  0.06102782, -0.26894492,  0.22315185,  0.2154729 ],
       [-0.41282264, -0.48822256, -0.01144446, -0.35066719, -0.36464645],
       [-0.22517599, -0.3051391 , -0.44061189,  0.51006137,  0.38122129],
       [ 0.15011733,  0.33565301,  0.24605599, -0.06375767, -0.11602387],
       [-0.13135266,  0.24411128, -0.01144446,  0.03187884,  0.2154729 ],
       [-0.03752933, -0.03051391,  0.07438902, -0.06375767,  0.13259871],
       [ 0.33776398,  0.06102782,  0.24605599, -0.15939418, -0.19889806],
       [-0.03752933, -0.12205564, -0.44061189,  0.41442486,  0.29834709],
       [-0.22517599,  0.24411128, -0.09727795,  0.12751534,  0.2154729 ],
       [-0.31899932, -0.3051391 ,  0.07438902, -0.35066719, -0.36464645],
       [ 0.15011733,  0.33565301,  0.1602225 , -0.06375767,  0.04972452],
       [ 0.15011733,  0.15256955, -0.3547784 ,  0.31878836,  0.29834709],
       [-0.22517599, -0.39668083,  0.1

In [111]:
# Проверяем корректность центрировки и нормировки
assert(np.allclose(X_cleaned, X_cleaned - X_cleaned.mean(axis=0))) # центр должен быть в нуле
assert(np.allclose(np.linalg.norm(X_cleaned, axis=0), np.array([1] * X_cleaned.shape[1]))) # l2 норма должна быть равна 1

## SVD разложение

Далее проведём разложение полученных данных для того, чтобы проанализировать данные на наличие скрытых признаков, на выявление главных признаков и зависимости между скрытыми признаками и изначальными

In [112]:
# Раскладываем
U, S, Vt = np.linalg.svd(X_cleaned, full_matrices=False)
S_r = np.diag(S)

In [113]:
print(f"U shape = {U.shape}")
print(f"S shape = {S.shape}")
print(f"Vt shape = {Vt.shape}")

U shape = (15, 5)
S shape = (5,)
Vt shape = (5, 5)


In [139]:
# Проверяем корректность разложения
assert(np.allclose(X_cleaned, U @ S_r @ Vt))

In [138]:
# Смотрим сингулярные числа
print(f"Сингулярные числа:\n{S}")
print(f"Квадраты сингулярных чисел:\n{S**2}")
print(f"Вклад факторов в вариацию данных (в %):")
contrib = S**2 / np.sum(S**2) * 100
print(contrib)

Сингулярные числа:
[1.65969941 1.33224208 0.61407764 0.25488052 0.16874093]
Квадраты сингулярных чисел:
[2.75460213 1.77486895 0.37709134 0.06496408 0.0284735 ]
Вклад факторов в вариацию данных (в %):
[55.09204252 35.49737894  7.54182686  1.29928162  0.56947006]


In [148]:
# Ищем первые k самых важных факторов
k = 2
sum_contrib = np.sum(contrib[:k])

X_k = U[:, :k] @ np.diag(S[:k]) @ Vt[:k, :]
R = X_cleaned - X_k
frobenius_error = np.sqrt(np.sum(R**2))
matrix_error = np.sqrt(np.linalg.svd(R, compute_uv=False)[0]**2)
print(f"Суммарный вклад первых {k} факторов: {sum_contrib:.2f}")
print(f"Ошибка аппроксимации (в норме Фробениуса): {frobenius_error:.2f}")
print(f"Ошибка аппроксимации (в матричной форме): {matrix_error:.2f}")

Суммарный вклад первых 2 факторов: 90.59
Ошибка аппроксимации (в норме Фробениуса): 0.69
Ошибка аппроксимации (в матричной форме): 0.61


Дальнейший анализ показал, что в данных:
1) Существует 2 скрытых фактора, которые суммарно имеют вклад 90.59% в данные 

## Анализ признаков

In [179]:
U2, S2, Vt2 = U[:, :k], S[:k], Vt[:k, :]
print(U2.shape)
print(S2.shape)
print(Vt2.shape)

(15, 2)
(2,)
(2, 5)


In [184]:
# Находим коэффициенты корреляции между скрытыми признаками и изначальными
feat_coeffs = Vt2.T * S2
feat_coeffs

array([[-0.29528239, -0.85319934],
       [ 0.04764026, -0.91990824],
       [-0.93759902, -0.2775059 ],
       [ 0.94964243, -0.22798311],
       [ 0.94033407, -0.26777388]])

Тут мы наблюдаем следующее:
1) Признаки 1 и 2 имеют заметную корреляцию (>85%) со вторым скрытым признаком. Логически можно предположить, что эти два вопроса объеденены общей тематикой "Отношение к банкам разных размеров". Далее, обращая внимание на знак и на значения в оригинальной таблице, можно предположить, что искомый признак может иметь название "Отношение к крупным банкам", где 0 -- крайне положительно, 9 -- крайне негативно.
2) Признаки 3, 4 и 5 имеют общую тематику "Отношение к обслуживанию" и сильно коррелируют с первым признаком. Соответственно, можно предположить, что название признака звучит как "Чувствительность к качеству обслуживания", где 0 -- очень чувствителен, 9 -- совершенно не требователен.

In [200]:
def translate_to_another_scale(orig_table: np.ndarray):
    table = copy.deepcopy(orig_table)
    for col in range(table.shape[1]):
        x_min, x_max = table[:, col].min(), table[:, col].max()
        if x_max == x_min:
            table[:, col] = np.zeros_like(table[:, col], dtype=int)
        table[:, col] = np.rint(9 * (table[:, col] - x_min) / (x_max - x_min)).astype(int)
    return table

In [ ]:
# Возвращаем новую, более компактную таблицу
X_new = translate_to_another_scale(U2)
X_new

array([[0., 0.],
       [6., 3.],
       [1., 9.],
       [9., 5.],
       [2., 0.],
       [5., 2.],
       [4., 3.],
       [1., 1.],
       [8., 4.],
       [6., 2.],
       [1., 7.],
       [3., 0.],
       [7., 1.],
       [1., 7.],
       [0., 0.]])